In [ ]:
# import libraries
import sys
import os
import pandas as pd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
from compositionspace.io import get_reconstructed_positions, get_iontypes, get_iontypes_info
from compositionspace.segmentation import ProcessSegmentation

In [ ]:
# ! pip list  
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

In [ ]:
RECONSTRUCTION_AND_RANGING = ("R21_08680-v02.pos", "R21_08680.rrng")
RECONSTRUCTION_AND_RANGING = ("PARAPROBE.Transcoder.Results.SimID.636502001.nxs",
                              "PARAPROBE.Ranger.Results.SimID.636502001.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.nxs"

In [ ]:
segmentation = ProcessSegmentation(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
segmentation.perform_pca_and_write_results()
segmentation.perform_bics_minimization_and_write_results()

In [ ]:
get_file_size(results_file_path)
H5Web(results_file_path)

<div class="alert alert-block alert-danger">
Discussion points:<br>
* Modify NXapm_composition_space<br>
* What to show how to show?<br>
* Number of cluster vs Number of clusters? wording...?<br>
* Why to run the gm several times, ones in the bics loop ones in get composition cluster files 
</div>